In [1]:
import mne
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Dense, Dropout, Flatten, Conv1D, 
                                     MaxPooling1D, GlobalAveragePooling1D)


import itertools

from IPython.utils import io

from ast import literal_eval

2022-07-19 15:27:11.209845: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-19 15:27:11.209884: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Purpose of this notebook

In this notebook we will be testing and creating the highest-performing individualized CNN models we can based on session 1 data, which will then be used as one of the L1 models in my final ensemble model to make our predictions on session 2 data.

### Import our data with the data ingester

I built a script to do this ingestion - please see the script file in the project repo for code.

After running, this script will load several dictionaries into memory, as well as other needed objects:
- data_dict - containing all the sample eeg data
- event_dict - which indicates the sample number at which each stimulus was applied
- y_dict - which has the type of experiment conducted in each trial
- info - file used to create MNE raw objects including channel names, type, and sampling frequency
- events_explained - dictionary which provides the names for each of the five trial types
- ch_names - list of all channel names

In [2]:
%run data_ingester.py

### Create initial gridsearch to look for best signal processing parameters to use with our CNN models

To begin, we'll start by importing our results from the CSP notebook. We need to use the same MNE parameters for all of our L1 models, so we'll plan to use all those settings with our CNN models.

In order to avoid models that are too highly fit, I'll be looking to create two CNN models per subject. Both will be shallow models, and I will aim for one CNN model to identify larger features in the EEG data, and one to capture smaller features.

Let's do a set of testing with that aim, and potentially another set thereafter if needed. In general, our goal is to create models with 75-80% accuracy on the test portion of our session 1 data, in the hopes those will be more generalizable models to bring into our ensemble.

After that, we'll examine how many epochs we can run those models for before overfitting begins to occur, and that number of epochs to run will become the final parameter of our individualized CNN models.

In [3]:
#Import final CSP model params to use as starting point
locked_params = pd.read_csv('data/csp_models_not_overfit.csv')

#Drop CSP variables not needed here, plus output column
locked_params.drop(['n_components', 'cov_est', 'log', 'test_acc'],
                   axis=1,
                  inplace=True)

#Reset trial combo to tuple, flat and reject to dicts
locked_params['trial_combo'] = (locked_params['trial_combo'].
                                apply(lambda x: literal_eval(x)))
#Use list comprehension b/c NaN can appear in list
locked_params['flat'] = [None if pd.isna(x) else literal_eval(x)
                         for x in locked_params['flat']]
#Use list comprehension b/c NaN can appear in list
locked_params['reject'] = [None if pd.isna(x) else literal_eval(x)
                         for x in locked_params['reject']]

locked_params['projectors_to_apply']

0    slice(None, 1, None)
1    slice(None, 1, None)
2    slice(None, 1, None)
3       slice(1, 2, None)
4    slice(None, 1, None)
5    slice(None, 1, None)
6    slice(None, 1, None)
7    slice(None, 1, None)
8       slice(1, 2, None)
Name: projectors_to_apply, dtype: object

In [4]:
#Manually overwrite projectors col, literal_eval doesn't work on slices
locked_params['projectors_to_apply'] = [slice(None, 1, None),
                                        slice(None, 1, None),
                                        slice(None, 1, None),
                                        slice(1, 2, None),
                                        slice(None, 1, None),
                                        slice(None, 1, None),
                                        slice(None, 1, None),
                                        slice(None, 1, None),
                                        slice(1, 2, None)]

In [9]:
#Shallow CNN with fewer, larger filters
sample_weight_options = [2, 3]
#First Conv1D layer settings
input_filter_count_options = [20, 30]
input_kernel_size_options = [10, 15]
input_strides_options = [6, 8]
#Hidden Conv1D layer settings
hidden_filters_options=[20, 30]
hidden_kernel_size_options=[7] 
hidden_strides_options=[7]
#Pooling layers settings
pool_size_options=[4]
#Penultimate dense layer settings
nodes_options = [20, 30]
model_type_options = ['Large_filters']


columns = ['sample_weight',
           'input_filter_count',
           'input_kernel_size',
           'input_strides',
           'hidden_filters',
           'hidden_kernel_size',
           'hidden_strides',
           'pool_size',
           'nodes',
           'model_type']

#Get number of permutations of variable parameters
permutations_1 = len(list(itertools.
                        product(sample_weight_options, 
                                input_filter_count_options,
                                input_kernel_size_options,
                                input_strides_options,
                                hidden_filters_options,
                                hidden_kernel_size_options,
                                hidden_strides_options,
                                pool_size_options,
                                nodes_options,
                                model_type_options)))

#Create dataframe of our first set of variable params
variable_params = pd.DataFrame(itertools.
                               product(sample_weight_options, 
                                input_filter_count_options,
                                input_kernel_size_options,
                                input_strides_options,
                                hidden_filters_options,
                                hidden_kernel_size_options,
                                hidden_strides_options,
                                pool_size_options,
                                nodes_options, 
                                model_type_options), 
                           columns=columns)

#Shallow CNN with more, smaller filters
sample_weight_options_2 = [2, 3]
#First Conv1D layer settings
input_filter_count_options_2 = [45, 55]
input_kernel_size_options_2 = [3, 6]
input_strides_options_2 = [1, 2]
#Hidden Conv1D layer settings
hidden_filters_options_2 = [45, 55]
hidden_kernel_size_options_2 = [3] 
hidden_strides_options_2 = [2]
#Pooling layers settings
pool_size_options_2 = [2]
#Penultimate dense layer settings
nodes_options_2 = [45, 50]
model_type_options_2 = ['Small_filters']

#Get number of permutations of variable parameters
permutations_2 = len(list(itertools.
                        product(sample_weight_options_2, 
                                input_filter_count_options_2,
                                input_kernel_size_options_2,
                                input_strides_options_2,
                                hidden_filters_options_2,
                                hidden_kernel_size_options_2,
                                hidden_strides_options_2,
                                pool_size_options_2,
                                nodes_options_2,
                                model_type_options_2)))

#Create dataframe of our first set of variable params
variable_params_2 = pd.DataFrame(itertools.
                               product(sample_weight_options_2, 
                                input_filter_count_options_2,
                                input_kernel_size_options_2,
                                input_strides_options_2,
                                hidden_filters_options_2,
                                hidden_kernel_size_options_2,
                                hidden_strides_options_2,
                                pool_size_options_2,
                                nodes_options_2,
                                model_type_options_2), 
                           columns=columns)

#Duplicate each row of our locked params to match the number of variable
#permutations we are going to run
locked_temp = pd.DataFrame(np.repeat(locked_params.values, 
                                     permutations_1+permutations_2, 
                                     axis=0))
locked_temp.columns = locked_params.columns




#Concat variable param grids together, and then on top of themselves
#Nine times to get data properly formatted
variable_params = pd.concat((variable_params, variable_params_2),
                              ignore_index=True)
variable_params = pd.concat((variable_params, 
                             variable_params,
                             variable_params,
                             variable_params,
                             variable_params,
                             variable_params,
                             variable_params,
                             variable_params,
                             variable_params),
                              ignore_index=True)

#Join our locked and variable parameters
test_df = locked_temp.join(variable_params)

#Add our column for test accuracy
test_df['val_acc'] = None

#Resent NaNs to None so MNE can read
test_df.replace(np.nan, None, inplace=True)

test_df.shape

(1152, 27)

### Creating our function to iterate across our test dataframes

In [7]:
def cnn_grid_search(test_df, 
                    savefile):
    """A function to iterate across a test dataframe and fill in the score on
    test data in the results column.
    
    All tests /rows must be at the individual subject level.
    
    Savefile is the filename to use in saving test dataframe to csv in data 
    directory each iteration."""
    for row in range(test_df.shape[0]):
        #Load each sessions data into an MNE raw object
        raw_dict = {}
        for key, value in data_dict.items():
            if 'sesh_1' in key:
                raw_dict[key] = mne.io.RawArray(value.T, info, verbose=0)

        #Filter data with bandpass. Note raw.filter applies in place
        for key, value in raw_dict.items():
            value.filter(l_freq=test_df.l_freq_filter[row], 
                         h_freq=test_df.h_freq_filter[row], 
                         method='fir', phase='zero', verbose=0)

        #Create epoch object with our raw objects and events arrays
        channels_to_keep = [ch for ch in ch_names if 
                            ch not in test_df.channels_to_drop[row]]
        epoch_dict = {}
        for key, value in raw_dict.items():
            epoch_dict[key] = mne.Epochs(value, events=event_dict[key], 
                                        event_id=events_explained, 
                                        tmin=-3, tmax=test_df.tmax[row], 
                                        baseline=test_df.baseline_correction[row],
                                        preload=True,
                                        picks=channels_to_keep, verbose=0,
                                        detrend=test_df.detrend[row],
                                        reject=test_df.reject[row],
                                        flat=test_df.flat[row],
                                        reject_tmin=test_df.tmin[row],
                                        reject_tmax=test_df.tmax[row])

        #Skip creating projectors step to save compute time if not being
        #applied in this iteration
        if test_df.projectors_to_apply[row]:
            #Create dictionary signal space projection vectors for each epoch
            proj_dict = {}
            for key, value in epoch_dict.items():
                proj_dict[key] = mne.compute_proj_epochs(value, 
                                                         n_eeg=2, 
                                                         verbose=0)
            #apply projectors
            for key, value in epoch_dict.items():
                value.add_proj(proj_dict[key][test_df.projectors_to_apply[row]], 
                               verbose=0)
                value.apply_proj(verbose=0)

        #Skip creating ICA components step to save compute time if not
        #being applied in this iteration
        if test_df.ica_to_exclude[row]:
            #create and fit ICA object to epochs
            for key, value in epoch_dict.items():
                ica = mne.preprocessing.ICA(n_components=5, method='picard', 
                                            max_iter='auto', verbose=0)
                ica.fit(value, verbose=0)
                #Apply the ICA
                ica.apply(value, exclude=test_df.ica_to_exclude[row],
                         verbose=0)

        #Resample the data at a new frequency, happens inplace
        for key, value in epoch_dict.items():
            value.resample(sfreq=test_df.selected_frequency[row])

        #Extract and standard scale data from all non-dropped epochs
        #Creates intermediate data dictionary
        int_data_dict = {}
        #Use robust sklearn scaler
        if test_df.scaler[row] == 'robust':
            mne_scaler = mne.decoding.Scaler(scalings='median')
            for key, value in epoch_dict.items():
                #with scalings=median implements sklearn robust scaler
                int_data_dict[key] = (mne_scaler.
                                      fit_transform(value.
                                                    get_data(tmin=test_df.tmin[row], 
                                                             tmax=test_df.tmax[row])))
        #No scaling option
        if test_df.scaler[row] is None:
            for key, value in epoch_dict.items():
                int_data_dict[key] = value.get_data(tmin=test_df.tmin[row], 
                                                      tmax=test_df.tmax[row])

        #Create updated dictionary of y values to reflect dropped epochs
        int_y_dict = {}
        for key, value in y_dict.items():
            if 'sesh_1' in key:
                temp_y_list = []
                for i, epoch in enumerate(epoch_dict[key].drop_log):
            #MNE drop log shows empty parens for epochs that were not dropped - 
            #these are the trials we are keeping in each iteration
                    if epoch == ():
                        temp_y_list.append(value[i])
                int_y_dict[key] = temp_y_list

        #Assemble final y dict with only trials in our current combo
        #In each combo, coding 1st trial type to 0, 2nd trial type to 1
        final_y_dict = {}
        for key, value in int_y_dict.items():
            temp_y_list = []
            for y in value:
                if y == test_df.trial_combo[row][0]:
                    temp_y_list.append(0)
                if y == test_df.trial_combo[row][1]:
                    temp_y_list.append(1)
            final_y_dict[key] = np.array(temp_y_list)

        #Assemble data dict with only trials in our current combo
        final_data_dict = {}
        for key, value in int_data_dict.items():
            index_list = []
            for i, y in enumerate(int_y_dict[key]):
                if (y == test_df.trial_combo[row][0] or 
                    y == test_df.trial_combo[row][1]):
                    index_list.append(i)
            final_data_dict[key] = value[index_list]

        #Train test split the data and y for the subject of current row
        for key, value in final_data_dict.items():
            if test_df.subject[row] in key:
                sub_X = value
        for key, value in final_y_dict.items():
            if test_df.subject[row] in key:
                sub_y = value
        (X_train, X_test, 
         y_train, y_test) = train_test_split(sub_X, 
                                             sub_y, 
                                             stratify=sub_y)
        
        #For subject, reset X and y in final dicts with train values,
        #test values will be passed to model for validation
        for key, value in final_data_dict.items():
            if test_df.subject[row] in key:
                final_data_dict[key] = X_train
        for key, value in final_y_dict.items():
            if test_df.subject[row] in key:
                final_y_dict[key] = y_train
        
        #Concatenate all X and y values together into X_train and y_train
        X_train = np.concatenate(([final_data_dict[key] for 
                                   key in final_data_dict.keys()]), axis=0)
        y_train = np.concatenate(([final_y_dict[key] for 
                                   key in final_y_dict.keys()]))
        
        
        #Create sample weight lists, higher weight for subject
        sample_weights = []
        for key, value in final_y_dict.items():
            if test_df.subject[row] in key:
                temp = [test_df.sample_weight[row]] * len(value)
                sample_weights = sample_weights + temp
            else:
                temp = [1] * len(value)
                sample_weights = sample_weights + temp
        
        #All data in y_test (our val data) is from the subject and
        #assigned our test sample weight
        val_sample_weights = [test_df.sample_weight[row]] * len(y_test)
        
        
        
        #Reshape all data tensors to feed into neural network
        X_train = np.reshape(X_train, (X_train.shape[0],
                                       X_train.shape[2],
                                       X_train.shape[1]))
        X_test = np.reshape(X_test, (X_test.shape[0],
                                     X_test.shape[2],
                                     X_test.shape[1]))
        y_train = np.reshape(y_train, 
                             len(y_train))
        y_test = np.reshape(y_test, 
                            len(y_test))
        sample_weights = np.reshape(sample_weights, 
                                    len(sample_weights))
        val_sample_weights = np.reshape(val_sample_weights, 
                                        len(val_sample_weights))


        #Suppress output
        with io.capture_output() as captured:
            #Model against our data and save the resulting scores
            model = Sequential()
            model.add(Conv1D(filters=test_df.input_filter_count[row], 
                             kernel_size=int(test_df.input_kernel_size[row]), 
                             strides=int(test_df.input_strides[row]), 
                             padding='same', 
                             activation='relu', 
                             input_shape=(X_train.shape[1], 
                                          X_train.shape[2])))
            model.add(Dropout(0.2))
            model.add(MaxPooling1D(int(test_df.pool_size[row])))
            model.add(Conv1D(filters=test_df.hidden_filters[row], 
                             kernel_size=int(test_df.hidden_kernel_size[row]), 
                             strides=int(test_df.hidden_strides[row]), 
                             padding='same', 
                             activation='relu'))
            model.add(Dropout(0.2))
            model.add(MaxPooling1D(int(test_df.pool_size[row])))
            model.add(GlobalAveragePooling1D())
            model.add(Dense(test_df.nodes[row], activation='relu'))
            model.add(Dropout(0.2))
            model.add(Dense(1, activation='sigmoid'))

        #Suppress output
        with io.capture_output() as captured:
            #compile & fit model
            model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  weighted_metrics=['accuracy'])
        #Suppress output
        with io.capture_output() as captured:
            history = model.fit(x=X_train, 
                                y=y_train, 
                                sample_weight=sample_weights,
                                batch_size=60, epochs=3, 
                                validation_data=(X_test, 
                                                 y_test,
                                                 val_sample_weights), 
                                verbose=0, workers=8)

        #Save the best val and training accuracy achieved by each model
        test_df.at[row, 'val_acc'] = max(history.history['val_accuracy'])
        test_df.at[row, 'train_acc'] = max(history.history['accuracy'])

        test_df.to_csv(f'data/{savefile}.csv', index=False)
        if row % 50 == 0:
            print(f'Grid search complete through row {row} of {test_df.shape[0]}')

### Now let's run our tests. See the bottom of this notebook for the CNN gridsearch function

In [12]:
#Run the test
cnn_grid_search(test_df=test_df, 
                savefile='L1_CNN_gridsearch_1')

Grid search complete through row 0 of 1152
Grid search complete through row 50 of 1152
Grid search complete through row 100 of 1152
Grid search complete through row 150 of 1152
Grid search complete through row 200 of 1152
Grid search complete through row 250 of 1152
Grid search complete through row 300 of 1152
Grid search complete through row 350 of 1152
Grid search complete through row 400 of 1152
Grid search complete through row 450 of 1152
Grid search complete through row 500 of 1152
Grid search complete through row 550 of 1152
Grid search complete through row 600 of 1152
Grid search complete through row 650 of 1152
Grid search complete through row 700 of 1152
Grid search complete through row 750 of 1152
Grid search complete through row 800 of 1152
Grid search complete through row 850 of 1152
Grid search complete through row 900 of 1152
Grid search complete through row 950 of 1152
Grid search complete through row 1000 of 1152
Grid search complete through row 1050 of 1152
Grid search

In [61]:
#view our results per subject - large filters
with pd.option_context('display.max_columns', None):
    display(test_df.loc[(test_df['subject'] == 'sub_L') & 
                        (test_df['model_type'] == 'Large_filters')].
            sort_values('val_acc', ascending=False).head(10))

,subject,trial_combo,flat,reject,baseline_correction,detrend,ica_to_exclude,scaler,l_freq_filter,h_freq_filter,channels_to_drop,projectors_to_apply,selected_frequency,tmin,tmax,train_acc,sample_weight,input_filter_count,input_kernel_size,input_strides,hidden_filters,hidden_kernel_size,hidden_strides,pool_size,nodes,model_type,val_acc
1058,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.555891,3,20,10,6,30,7,7,4,20,Large_filters,1.0
1043,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.565495,2,30,10,6,30,7,7,4,30,Large_filters,0.833333
1057,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.528701,3,20,10,6,20,7,7,4,30,Large_filters,0.833333
1049,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.550955,2,30,15,6,20,7,7,4,30,Large_filters,0.833333
1063,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.519637,3,20,10,8,30,7,7,4,30,Large_filters,0.833333
1071,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.531915,3,20,15,8,30,7,7,4,30,Large_filters,0.833333
1069,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.551515,3,20,15,8,20,7,7,4,30,Large_filters,0.833333
1037,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.530351,2,20,15,8,20,7,7,4,30,Large_filters,0.833333
1036,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.520767,2,20,15,8,20,7,7,4,20,Large_filters,0.833333
1055,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.561905,2,30,15,8,30,7,7,4,30,Large_filters,0.833333


In [62]:
#view our results per subject - small filters
with pd.option_context('display.max_columns', None):
    display(test_df.loc[(test_df['subject'] == 'sub_L') & 
                        (test_df['model_type'] == 'Small_filters')].
            sort_values('val_acc', ascending=False).head(10))

,subject,trial_combo,flat,reject,baseline_correction,detrend,ica_to_exclude,scaler,l_freq_filter,h_freq_filter,channels_to_drop,projectors_to_apply,selected_frequency,tmin,tmax,train_acc,sample_weight,input_filter_count,input_kernel_size,input_strides,hidden_filters,hidden_kernel_size,hidden_strides,pool_size,nodes,model_type,val_acc
1089,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.504732,2,45,3,1,45,3,2,2,50,Small_filters,0.833333
1097,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.507886,2,45,6,1,45,3,2,2,50,Small_filters,0.833333
1107,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.50625,2,55,3,1,55,3,2,2,50,Small_filters,0.833333
1105,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.534375,2,55,3,1,45,3,2,2,50,Small_filters,0.833333
1134,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.536443,3,45,6,2,55,3,2,2,45,Small_filters,0.666667
1133,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.55102,3,45,6,2,45,3,2,2,50,Small_filters,0.666667
1132,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.48688,3,45,6,2,45,3,2,2,45,Small_filters,0.666667
1131,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.51895,3,45,6,1,55,3,2,2,50,Small_filters,0.666667
1130,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.51312,3,45,6,1,55,3,2,2,45,Small_filters,0.666667
1088,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.507886,2,45,3,1,45,3,2,2,45,Small_filters,0.666667


**Based on the dataframe above, let's assemble our selections of final CNN models**

Looked through the list of output manually to find models that were very similar on train and test accuracy and 75-80% accurate, in both the larger and smaller models category.

In this case, train accuracy is the accuracy of the model against the entire dataset of session 1, less our small test split from the particular subject (since I fed it the whole dataset but added extra sample weight to our target candidate). My training accuracies as a result are generally quite low, and so I'm often moving forward with models that are only 55% accurate. Unsurprisingly, those models that are most broadly applicable tend to have a lower sample weight toward our target.

There are often cases where they do much better on the test data (which is just the subject in question). For now, however, I am not bringing those models into my final ensemble model, but I am going to save them down into an index list, and if my ensemble isn't performing well, they could be a good group to add to the mix. I am not pulling the highest performing models, again to avoid overfitting, but am picking those with approx 75-80% val accuracy for that list. Unfurprisingly, these have a higher weight toward our target.

In [64]:
high_train_acc_models = [12, 121, 
                         181, 238, 
                         312, 351, 
                         411, 462, 
                         569, 592, 
                         658, 752,
                         827, 832,
                         925, 988,
                         1072, 1151]
high_val_acc_models = [7, 103,
                      150, 216,
                      295, 378,
                      443, 483,
                      515, 631,
                      683, 735,
                      823, 881,
                      915, 974,
                      1043, 1105]

In [66]:
test_df.iloc[high_train_acc_models].to_csv('data/CNN_models_most_general.csv',
                                          index=False)

In [68]:
test_df.iloc[high_val_acc_models].to_csv('data/CNN_models_more_biased.csv',
                                          index=False)